<a href="https://colab.research.google.com/github/3zero69/-/blob/main/%ED%95%B8%EC%A6%88%EC%98%A8%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_CH7_%EC%95%99%EC%83%81%EB%B8%94%ED%95%99%EC%8A%B5%EA%B3%BC%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8_pg245_271.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#7. 앙상블 학습과 랜덤 포레스트

- **앙상블 학습** :  **앙상블** , 일련의 예측기(즉, 분류나 회귀모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있음


 - **앙상블 방법** : 앙상블 학습 알고리즘
    - 예)  **랜덤 포레스트** : 결정트리의 앙상블, 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킬 수 있음
      - 예측을 하려면 모든 개별 트리의 예측을 구하면 됨
      - 그런 다음 가장 많은 선택을 받은 클래스를 예측으로 삼음
  - 가장 인기있는 앙상블 방법: **배깅, 부스팅, 스태킹**

프로젝트의 마지막에 다다르면 흔히 앙상블 방법을 사용하여 이미 만든 여러 괜찮은 예측기를 연결하여 더 좋은 예측기를 만듦


## 7.1 투표 기반 분류기



- 정확도가 80% 인 분류기 여러 개를 훈련 (로지스틱 회귀 분류기, 랜덤 포레스트 분류기, K-최근접 이웃 분류기 등)

- 더 좋은 분류기를 만드는 매우 간단한 방법은 각 분류기의 예측을 모아 가장 많이 선택된 클래스를 예측하는 것
  - 다수결 투표로 정해지는 분류기 - **직접 투표 분류기**
  - 다수결 투표 분류기는 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다 정확도가 더 높음

- 각 분류기가 약한 학습기(랜덤 추측보다 약간 더 높은 성능을 내는 분류기) 이더라도 충분히 많고 다양하다면, 앙상블은 강한 학습기(높은 정확도를 내는 분류기) 가 될 수 있음

* 어떻게 ? 
  - 큰 수의 법칙 : 던진 횟수가 증가할수록 앞면이 나올 확률이 51% 에 가까워짐
- 이와 비슷하게 51% 정확도(무작위 추측보다 조금 더 나은) 를 가진 1,000 개의 분류기로 앙상블 모델을 구축
  - 가장 많은 클래스를 예측으로 삼으면 75% 정확도 기대 가능
  - 물론 모든 분류기가 완벽히 독립적이고 오차에 상관관계가 없는 경우에 성립

In [1]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y=make_moons(n_samples=500, noise=0.15)
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2)

In [2]:
#@title 여러 분류기를 조합한 투표 기반 분류기 (VotingClassifier) 훈련 코드
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
# 각 분류기의 테스트셋 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

  ## 투표 기반 분류기가 다른 분류기보다 성능이 조금 더 높음

LogisticRegression 0.91
RandomForestClassifier 0.99
SVC 0.99
VotingClassifier 0.99


모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_proba() 메서드), 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스 예측 가능 - 간접 투표

확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높음

- voting = "hard" 를 voting = "soft" 로 바꾸고, 모든 분류기가 클래스의 확률을 추정할 수 있으면 됨
- SVC : probabilty = True 지정

## 7.2 배깅과 페이스팅

다양한 분류기를 만드는 한가지 방법 : 각기 다른 훈련 알고리즘을 사용하는 것

또 다른 방법 : 같은 알고리즘을 사용하고, 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습

* 배깅 : 훈련 세트에서 중복을 허용하여 샘플링
* 페이스팅 : 중복을 허용하지 않고 샘플링
  - 배깅과 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용 가능
  - 배깅만 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링 가능

- 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모다 새로운 샘플에 대한 예측을 만듦
  - 수집 함수: 전형적으로 분류일때는 **통계적 최빈값** 이고, 회귀에 대해서는 평균을 계산

- 개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만, 수집 함수를 통과하면 편향과 분산이 모두 감소

- 앙상블 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 줄어듦

### 7.2.1 사이킷런의 배깅과 페이스팅

In [4]:
#@title 결정 트리 분류기 500개의 앙상블 훈련 코드

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1)    
# 각 분류기는 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련 (배깅의 경우)
# 페이스팅 사용시, bootstrap = False 지정
# n_jobs : 사이킷런이 훈련과 예측에 사용할 CPU 코어 수 (-1 : 가용한 모든 코어 사용)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

앙상블의 예측이 결정 트리 하나의 예측보다 일반화가 훨씬 잘됨 

 앙상블은 비슷한 편향에서 더 작은 분산을 만듦(오차 수 거의 비슷, 결정 경계 덜 불규칙)

### 7.2.2 oob 평가

배깅 사용시, 어떤 샘플은 한 예측기를 위해 여러번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있음
  - BaggingClassifier 는 기본값으로 중복을 허용하여 훈련 세트의 크기만큼인 m 개의 샘플을 선택
  - 이는 평균적으로 각 예측기에 훈련 샘플의 63% 정도만 샘플링 된다는 것을 의미

선택되지 않은 훈련 샘플의 나머지 73% 를 obb 샘플 이라고 부름 (예측마다 남겨진 37% 는 모두 다름)

- 예측기 훈련 동안에는 obb 샘플을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가 가능
  - 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻음

In [5]:
#@title 훈련이 끝나고 자동으로 oob 평가 수행 (oob_score = True)

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_  ## 테스트 세트에서 약 0.9725 의 정확도를 얻을 것으로 보임

0.9825

In [7]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)  ## 실제로 그러함 (비슷하군)

0.99

In [8]:
# oob 샘플에 대한 결정 함수의 값 확인
bag_clf.oob_decision_function_ 

## 첫번째 훈련 샘플이 양성 클래스에 속할 확률 0.01030928 로 추정 (음성 클래스 0.98969072)

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.20218579, 0.79781421],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.96470588, 0.03529412],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.19402985, 0.80597015],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.49152542, 0.50847458],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.04324324, 0.95675676],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

	BaggingClassifier
-	Max_features
-	Bootstrap_features 
-	Max_samples, bootstrap 과 동일, 샘플이 아닌 특성이 대한 샘플링
-	각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련
-	매우 고차원의 데이터셋을 다룰 때 
 
	랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링 하는 것
-	랜덤 서브 스페이스방식 : 훈련 샘플을 모두 사용하고 특성은 샘플링하는 것


##7.4 랜덤 포레스트

랜덤 포레스트 : 일반적으로 배깅 방법(또는 페이스팅) 을 적용한 결정 트리 앙상블



In [9]:
#@title 랜덤 포레스트 훈련 

from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1) # 500 개 트리로 이루어진 랜덤 포레스트 분류기를 여러 CPU 코어에서 훈련
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

랜덤포레스트 알고리즘은 트리의 노드를 분할할 때 전체 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성을 더 주입함 

이는 결국 트리를 더욱 다양하게 만들고 편향을 손해보는 대신 분산을 낮춤 (훌륭한 모델)

In [10]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)

### 7.4.1 엑스트라 트리

랜덤 포레스트
-	트리를 만들 때 각 노드는 무작위로 특성의 서브셋을 만들어 분할에 사용
-	트리를 더욱 무작위로 만들기 위해 최적의 임곗값을 찾는 대신 후보 특성을 사용해 무작위로 분할한 다음 그 중에서 최상의 분할을 선택

**익스트림 랜덤 트리 앙상블 (엑스트라 트리)** : 극단적으로 무작위한 트리의 랜덤 포레스트
-	편향이 늘어나지만 대신 분산을 낮춤
-	모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이므로 일반적인 랜덤 포레스트보다 엑스트라가 훨씬 빠름


### 7.4.2 특성 중요도

랜덤 포레스트 
-	특성의 상대적 중요도를 측정하기 쉬움
-	어떤 특성을 사용한 노드가 (모든 트리에 걸쳐서) 평균적으로 불순도를 얼마나 감소시키는지 확인하여 특성의 중요도를 측정
	가중치 평균이며 각 노드의 가중치는 연관된 훈련 샘플 수와 같음


랜덤 포레스트는 특히 특성을 선택 해야할 때 어떤 특성이 중요한지 빠르게 확인 할 수 있어 매우 편리함


In [11]:

from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
  print(name, score)
  
## 가장 중요한 특성은 꽃잎의 길이(44%) 와 너비(42%) 이고, 꽃받침의 길이와 너비는 비교적 덜 중요해 보임

sepal length (cm) 0.10296557698178051
sepal width (cm) 0.024478156439106512
petal length (cm) 0.4277171936623511
petal width (cm) 0.44483907291676184


## 7.5 부스팅

* **부스팅** : 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법

 - 앞의 모델을 보완해나가면서 일련의 예측기를 학습
  - 에이다부스트 
  - 그레이디언트 부스트

### 7.5.1 에이다부스트

In [12]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

###7.5.2 그레이디언트 부스팅

In [13]:
# DecisionTreeRegressor를 훈련 세트에 학습
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [14]:
# 첫 번째 예측기에서 생긴 잔여 오차에 두 번째 DecisionTreeRegressor 훈련
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2)

In [15]:
# 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델 훈련
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2)

In [17]:
# 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 됨
# y_pred = sum(tree.predict(X_new)) for tree in (tree_reg1, tree_reg2, tree_reg3)

In [18]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import numpy as np

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=120)

In [22]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train, y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break    # 조기 종료

In [23]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)
     

# 자동 조기 종료와 같은 여러 좋은 기능도 제공
xgb_reg.fit(X_train, y_train,
            eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)
     

[0]	validation_0-rmse:0.36415
[1]	validation_0-rmse:0.27083
[2]	validation_0-rmse:0.20305
[3]	validation_0-rmse:0.16171
[4]	validation_0-rmse:0.13570
[5]	validation_0-rmse:0.12274
[6]	validation_0-rmse:0.11789
[7]	validation_0-rmse:0.11750
[8]	validation_0-rmse:0.12038


/usr/local/lib/python3.9/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


## 7.6 스태킹

**스태킹** : 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수( like 직접 투표 ) 대신 취합하는 모델을 훈련 시키는 방법에 대한 고안
 - 새로운 샘플에 회귀 작업을 수행하는 앙상블
 - 예측기는 각각 다른 값을 예측하고 마지막 예측기(블렌더 또는 메타 학습기) 가 이 예측을 입력으로 받아 최종 예측을 만듦
  - 블렌더를 학습시키는 일반적인 방법: 홀드 아웃 세트 사용 
  1. 훈련 세트를 두 개의 서브셋으로 나눔
    - 첫번째 서브셋은 첫번째 레이어의 예측을 훈련시키기 위해 사용됨
  2. 첫번째 레이어의 예측기를 사용해 두번째 세트에 대한 예측을 만듦 
    - 예측기들이 훈련하는 동안 이 샘플들을 전혀 보지 못했기 때문에 이때 만들어진 예측은 완전히 새로운 것임
  3. 두번쨰 세트는 두번째 레이어를 훈련시키기 위한 훈련 세트를 만드는데 사용
  ...
  4. 작업이 끝나면 각 레이어를 차례대로 실행해서 새로운 샘플에 대한 예측을 만들 수 있음

